In [5]:
import pandas as pd
import numpy as np
import requests
import re
import time as tm
import json

###############   Url加载函数    #########################
#实现：通过request模拟请求获取url的页面html数据，这里是比较完整的一个url请求函数
#输入：url地址
#返回：解析后的url文本内容r.text
##########################################################
def fetchURL(url):
    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',
    }
    try:
        r = requests.get(url,headers=headers)  #模拟http请求
        r.raise_for_status()        #请求结果状态
        return r.text
    except requests.HTTPError as e:
        print(e)
        print("HTTPError")
    except requests.RequestException as e:
        print(e)
    except:
        print("Unknown Error !")
        
###############   页面数据爬取函数    #########################
#实现：通过json解析html文本，获取所需信息：从B站游戏评论中获取评论内容，评分，时间，赞/踩，用户名，用户等级
#输入：html文本
#返回：commentlist，包含10条上述结果的二维数组。（每个页面有10条评论）
##########################################################
def parserHtml(html):
    s = json.loads(html)  #用json解析html文本
    #准备容器
    content=[]
    grade=[]
    publish_time=[]
    up_count=[]
    down_count=[]
    user_name=[]
    user_level=[]
    #抓取每一条评论记录的所需信息
    for i in range(10):
        try:
            comment = s['data']['list'][i]
            content.append(comment['content'])
            grade.append(comment['grade'])
            publish_time.append(comment['publish_time'])
            up_count.append(comment['up_count'])
            down_count.append(comment['down_count'])
            user_name.append(comment['user_name'])
            user_level.append(comment['user_level'])
        except:
            break
    #将本页面的10条记录合并保存下来，变成一个二维表格
    commentlist=[content,
                 grade,
                 publish_time,
                 up_count,
                 down_count,
                 user_name,
                 user_level]
    return commentlist      #返回这个页面的10条评论

#***********主程序************
#step 1 输入需要爬取的b站游戏的信息

app_id="104656"    #咔叽的B站ID，每个游戏都有一个   正式服103110  测试服104656
page_total=50          #需要爬取的总页数

#准备容器
all_content=[]
all_grade=[]
all_publish_time=[]
all_up_count=[]
all_down_count=[]
all_user_name=[]
all_user_level=[]

print('*************************************************')
print('Grapping comment data of app:{0} from BiliBili...'.format(app_id))
print('*************************************************\n')

#step 2  爬取数据，循环爬取，每页获取10条评论数据
for pagenum in range(1,page_total+1):
    t1=tm.time()  #用于调试代码效率
    url="https://line1-h5-pc-api.biligame.com/game/comment/page?game_base_id={0}&rank_type=2&page_num={1}&page_size=10".format(app_id, pagenum)
    comment_page = fetchURL(url)  #加载url
    parsered_comment=parserHtml(comment_page) #解析一页
    
    #从解析后的json对象中获取所需要的信息
    all_content.extend(parsered_comment[0])
    all_grade.extend(parsered_comment[1])
    all_publish_time.extend(parsered_comment[2])
    all_up_count.extend(parsered_comment[3])
    all_down_count.extend(parsered_comment[4])    
    all_user_name.extend(parsered_comment[5])
    all_user_level.extend(parsered_comment[6])
    t2=tm.time()  #用于调试代码效率
    timing=t2-t1                                      
    print('Page %d grapped, %5.2f seconds used' % (pagenum,timing))  #输出爬取进度
    if pagenum%10 == 0:
            print('Taking a break... To avoid being detected')
            tm.sleep(5)
            
#step 3  爬取完成，整理并导出数据
result={"content" : all_content,
    "grade" : all_grade,
    "publish_time" : all_publish_time,
    "up_count" : all_up_count,
    "down_count" : all_down_count,
    "user_name" : all_user_name,
    "user_level" : all_user_level,
    } 
    
resultpd=pd.DataFrame(result)
print('\n*************************************************')
print('Comment grapping finished. %d comments grapped in total' % (len(resultpd['content'])))
resultpd.to_excel('Bili_comment_appid{}.xlsx'.format(app_id))
print('Written to 【Bili_comment_appid{}.xlsx】'.format(app_id))
print('*************************************************')


*************************************************
Grapping comment data of app:104656 from BiliBili...
*************************************************

Page 1 grapped,  0.23 seconds used
Page 2 grapped,  0.24 seconds used
Page 3 grapped,  0.28 seconds used
Page 4 grapped,  0.26 seconds used
Page 5 grapped,  0.45 seconds used
Page 6 grapped,  0.32 seconds used
Page 7 grapped,  0.27 seconds used
Page 8 grapped,  0.24 seconds used
Page 9 grapped,  0.28 seconds used
Page 10 grapped,  0.27 seconds used
Taking a break... To avoid being detected
Page 11 grapped, 21.36 seconds used
Page 12 grapped, 21.33 seconds used
Page 13 grapped,  0.23 seconds used
Page 14 grapped, 21.37 seconds used
Page 15 grapped,  0.28 seconds used
Page 16 grapped, 21.47 seconds used
Page 17 grapped,  0.26 seconds used
Page 18 grapped,  0.29 seconds used
Page 19 grapped,  0.23 seconds used
Page 20 grapped,  0.23 seconds used
Taking a break... To avoid being detected
Page 21 grapped,  0.27 seconds used
Page 22 grapped

In [2]:
resultpd.head(10)

,content,grade,publish_time,up_count,down_count,user_name,user_level
0,上个服我玩到了，这次怎么下不了了,6,2020-04-24 09:50:11,0,0,圣冕之月,5
1,上次测试充值不算了？ 还是这次测试不给？,10,2020-04-24 07:32:32,0,0,丨莉莉娅丨,5
2,有一说一，人刚卸载测试服，圈内人多，匿了\n总得说算是不肝，如果维持这个爆率，对新人也很有好...,8,2020-04-23 22:43:30,0,0,水源千鹤Official,5
3,挺好玩的，就是有点肝。,8,2020-04-23 20:49:34,0,0,达库-罗斯琳德,5
4,太肝了\n打到手机快炸了。只能存个进度\n\n1 咔叽界面显示技能太费劲了，长按还会跑偏。\...,10,2020-04-23 20:26:22,0,0,HIGHLLL,4
5,可可爱爱没有脑袋没有感到无聊就说明这个游戏已经做成功了，我觉得没有不好的地方,10,2020-04-23 20:10:40,0,0,玩胖次辣,4
6,1、希望能给参与测试的玩家一个专属的限定皮肤，毕竟我好舍不得我的小黄叽啊，555\n2、各个...,10,2020-04-23 18:17:11,0,0,孤寂深夜,5
7,希望移动那些小鸡的时候，屏幕不要突然间跳到别的地方去。\n发现剧情全推完了，希望到时候能增加...,10,2020-04-23 18:06:19,0,0,当个好人_,5
8,个人觉得没什么大毛病就是有个小bug：在咔叽上下电梯的时候把他们捉回上电梯的地方。次数多了，...,8,2020-04-23 17:59:12,0,0,风茗乐,5
9,画风好可爱阿！但是我没搞清楚怎么获得新叽。。导致十几级了9只上阵叽都没凑齐。还有不知道为什么...,10,2020-04-23 17:20:12,0,0,Clever1Body,5
